In [8]:
##Data Analysis: PEI Case study

import pandas as pd

#Load Customer Data for Analysis
customers_df = pd.read_excel("Customer.xls")
print(customers_df.head())
print(customers_df.info())

# Checking for the duplicates in  Customer_id's
duplicate_customers = customers_df[customers_df['Customer_ID'].duplicated()]

# Check for missing values in Customers data
print(customers_df.isnull().sum())

# Filling missing values with a default value for Country column
customers_df.fillna({'Country': 'Unknown'}, inplace=True)

#checking the Customers Dataframe for customers with Age > 120
invalid_ages = customers_df[~customers_df['Age'].between(0, 120)]

# Check if negative values are present in the Age column
negative_ages = customers_df[customers_df['Age'] < 0]
print(negative_ages)

   Customer_ID    First     Last  Age Country
0            1   Joseph     Rice   43     USA
1            2     Gary    Moore   71     USA
2            3     John   Walker   44      UK
3            4     Eric   Carter   38      UK
4            5  William  Jackson   58     UAE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  250 non-null    int64 
 1   First        250 non-null    object
 2   Last         250 non-null    object
 3   Age          250 non-null    int64 
 4   Country      250 non-null    object
dtypes: int64(2), object(3)
memory usage: 9.9+ KB
None
Customer_ID    0
First          0
Last           0
Age            0
Country        0
dtype: int64
Empty DataFrame
Columns: [Customer_ID, First, Last, Age, Country]
Index: []


In [10]:
orders_df = pd.read_csv('Order.csv')
print(orders_df.head())
print(orders_df.info())
print(orders_df.columns)


# Checking for the duplicates in  Order_id's
duplicate_Orders = orders_df[orders_df['Order_ID'].duplicated()]

# Checking for missing values in Orders data
print(orders_df.isnull().sum())

orders_df.fillna({ 'Amount': 0.0}, inplace=True)

# Check if there are any negative Amount 
negative_amounts = orders_df[orders_df['Amount'] < 0]
print(negative_amounts)


   Order_ID      Item  Amount  Customer_ID
0         1  Keyboard     400          139
1         2     Mouse     300          250
2         3   Monitor   12000          239
3         4  Keyboard     400          153
4         5  Mousepad     250          153
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     250 non-null    int64 
 1   Item         250 non-null    object
 2   Amount       250 non-null    int64 
 3   Customer_ID  250 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.9+ KB
None
Index(['Order_ID', 'Item', 'Amount', 'Customer_ID'], dtype='object')
Order_ID       0
Item           0
Amount         0
Customer_ID    0
dtype: int64
Empty DataFrame
Columns: [Order_ID, Item, Amount, Customer_ID]
Index: []


In [11]:
# Load Shippings Data in json format and normalise it to Analyse the Data 
import json
with open('Shipping.json') as f:
    shippings_data = json.load(f)
shippings_df = pd.json_normalize(shippings_data)
print(shippings_df.head())
print(shippings_df.info())

# Checking for the duplicates in Shipping_ids
duplicate_Shippings = shippings_df[shippings_df['Shipping_ID'].duplicated()]

# Check for missing values in Shippings data
print(shippings_df.isnull().sum())

shippings_df.fillna({'Status': 'Unknown'}, inplace=True)


#for ease of working..convert the json file to CSV
with open('Shipping.json', 'r') as json_file:
   json_data = json.load(json_file)
df = pd.json_normalize(json_data)
df.to_csv('Shipping.csv', index=False)




   Shipping_ID     Status  Customer_ID
0            1    Pending          173
1            2    Pending          155
2            3  Delivered          242
3            4    Pending          223
4            5  Delivered           72
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Shipping_ID  250 non-null    int64 
 1   Status       250 non-null    object
 2   Customer_ID  250 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB
None
Shipping_ID    0
Status         0
Customer_ID    0
dtype: int64


In [12]:
# checking if  all Customer_id's in Orders data exist in Customers
missing_customers = orders_df[~orders_df['Customer_ID'].isin(customers_df['Customer_ID'])]
print(missing_customers)
# checking for all the Customer_ids in Shippings exist in Customers
missing_customers_shipping = shippings_df[~shippings_df['Customer_ID'].isin(customers_df['Customer_ID'])]
print(missing_customers_shipping)


Empty DataFrame
Columns: [Order_ID, Item, Amount, Customer_ID]
Index: []
Empty DataFrame
Columns: [Shipping_ID, Status, Customer_ID]
Index: []


In [ ]:
#1.the total amount spent and the country for the Pending delivery status for each country


In [13]:

# Merge Orders and Shippings Data
merged_customers_orders_shipping = pd.merge(customers_df,orders_df,on='Customer_ID', how='inner')
Amount_spent_by_country=merged_customers_orders_shipping.groupby('Country')['Amount'].sum().reset_index()
print(Amount_spent_by_country)

#Merge Customers and Shipping
merged_customers_shipping=pd.merge(customers_df,shippings_df,on='Customer_ID', how='inner')
#Filtering where status is Pending
pending_deliveries = merged_customers_shipping[merged_customers_shipping['Status'] == 'Pending']
#Grouping by country 
Country_Pending_Delivery_count = pending_deliveries.groupby('Country')['Status'].count().reset_index()
#renaming Columns foir better Understanding
Country_Pending_Delivery_count=Country_Pending_Delivery_count.rename(columns={'Status':'Pending_Delivery_Count'})
print(Country_Pending_Delivery_count)

Final_Df=pd.merge(Amount_spent_by_country,Country_Pending_Delivery_count,on='Country',how='inner')

print(Final_Df)

  Country  Amount
0     UAE   49950
1      UK  244350
2     USA  238200
  Country  Pending_Delivery_Count
0     UAE                      39
1      UK                      59
2     USA                      52
  Country  Amount  Pending_Delivery_Count
0     UAE   49950                      39
1      UK  244350                      59
2     USA  238200                      52


In [ ]:
#2.the total number of transactions, total quantity sold, and total amount spent for each customer, along with the product details.


In [15]:

# Group by Customer_ID and aggregate the required information
customer_summary = orders_df.groupby('Customer_ID').agg({'Order_ID': 'count','Amount': 'sum'}).reset_index()

print(customer_summary.columns)

customer_summary = customer_summary.rename(columns={'Order_ID': 'total_transactions', 'Amount': 'total_amount_spent'})


# Merge with customer information
customer_summary = pd.merge(customer_summary, customers_df, on='Customer_ID', how='inner')

# Collecting product details for each customer as a list
product_details = orders_df.groupby('Customer_ID')['Item'].apply(', '.join).reset_index()
product_details.columns
product_details=product_details.rename(columns={'Item':'Items_list'})
print(product_details.head())


customer_summary = pd.merge(customer_summary, product_details, on='Customer_ID', how='inner')
customer_summary['Full Name']=customer_summary['First']+' '+customer_summary['Last']
customer_summary.columns
customer_summary = customer_summary.drop(columns=['First', 'Last','Country','Age'])
print(customer_summary.head())
print(customer_summary.columns)


Index(['Customer_ID', 'Order_ID', 'Amount'], dtype='object')
   Customer_ID                           Items_list
0            4                             Mousepad
1            5                              DDR RAM
2            8  Mousepad, DDR RAM, Webcam, Mousepad
3           10                             Keyboard
4           12                             Harddisk
   Customer_ID  total_transactions  total_amount_spent  \
0            4                   1                 200   
1            5                   1                1500   
2            8                   4                2300   
3           10                   1                 400   
4           12                   1                5000   

                            Items_list         Full Name  
0                             Mousepad       Eric Carter  
1                              DDR RAM   William Jackson  
2  Mousepad, DDR RAM, Webcam, Mousepad  Jason Montgomery  
3                             Keyboard    

In [ ]:
#3.the maximum product purchased for each country.

In [16]:

# Merge Orders with Customers to get the Country information
print(orders_df.head())
customers_df.head()
orders_with_customers = pd.merge(orders_df, customers_df[['Customer_ID', 'Country']], on='Customer_ID', how='inner')
print(orders_with_customers.head())

# Group by Country and Item and calculate the total quantity sold for each product in each country
product_quantity = orders_with_customers.groupby(['Country', 'Item']).agg({'Amount': 'sum'}).reset_index()
print(product_quantity.columns) #checking column names
product_quantity=product_quantity.rename(columns={'Amount':'total_amount'})

# Identify the maximum product purchased for each country
idx = product_quantity.groupby('Country')['total_amount'].idxmax()
max_product_per_country = product_quantity.loc[idx].reset_index(drop=True)

print(max_product_per_country)


   Order_ID      Item  Amount  Customer_ID
0         1  Keyboard     400          139
1         2     Mouse     300          250
2         3   Monitor   12000          239
3         4  Keyboard     400          153
4         5  Mousepad     250          153
   Order_ID      Item  Amount  Customer_ID Country
0         1  Keyboard     400          139      UK
1         2     Mouse     300          250     USA
2         3   Monitor   12000          239      UK
3       211  Keyboard     400          239      UK
4         4  Keyboard     400          153      UK
Index(['Country', 'Item', 'Amount'], dtype='object')
  Country      Item  total_amount
0     UAE  Harddisk         20000
1      UK   Monitor        156000
2     USA   Monitor        132000


In [ ]:
#4.the most purchased product based on the age category less than 30 and above 30.

In [17]:

# Create\ing age categories <30 and 30 or above
customers_df['AgeCategory'] = customers_df['Age'].apply(lambda x: '<30' if x < 30 else '>=30')

# Merging Orders with Customers including AgeCategory
orders_with_age_category = pd.merge(orders_df, customers_df[['Customer_ID', 'AgeCategory']], on='Customer_ID', how='inner')

# Group by AgeCategory and Item and calculate the total quantity sold for each product in each age category

product_quantity_age_cat= orders_with_age_category.groupby(['AgeCategory', 'Item']).agg({'Order_ID':'count'}).reset_index()
print(product_quantity_age_cat.columns)
#rENAMING Columns
product_quantity_age_cat=product_quantity_age_cat.rename(columns={'Order_ID':'Order_count'})

sorted_product_quantity = product_quantity_age_cat.sort_values(by=['AgeCategory', 'Order_count'], ascending=[True, False])
print(sorted_product_quantity)
max_product_per_age_category = sorted_product_quantity.drop_duplicates(subset=['AgeCategory'], keep='first').reset_index(drop=True)

print(max_product_per_age_category)

Index(['AgeCategory', 'Item', 'Order_ID'], dtype='object')
   AgeCategory      Item  Order_count
6          <30  Mousepad           17
3          <30  Keyboard           15
7          <30    Webcam            9
0          <30   DDR RAM            8
4          <30   Monitor            8
2          <30   Headset            7
5          <30     Mouse            7
1          <30  Harddisk            5
11        >=30  Keyboard           35
14        >=30  Mousepad           33
9         >=30  Harddisk           20
10        >=30   Headset           18
13        >=30     Mouse           18
8         >=30   DDR RAM           17
12        >=30   Monitor           17
15        >=30    Webcam           16
  AgeCategory      Item  Order_count
0         <30  Mousepad           17
1        >=30  Keyboard           35


In [ ]:
# Group by Country and calculate the total number of transactions and total amount spent

In [18]:

country_summary = orders_with_customers.groupby('Country').agg(total_transactions=('Order_ID', 'count'), total_amount_spent=('Amount', 'sum')
).reset_index()

print(country_summary.shape)


min_transactions_country = country_summary.sort_values(by='total_transactions').iloc[0]
print(min_transactions_country)

min_sales_country = country_summary.sort_values(by='total_amount_spent').iloc[0]
print(min_sales_country)


(3, 3)
Country                 UAE
total_transactions       40
total_amount_spent    49950
Name: 0, dtype: object
Country                 UAE
total_transactions       40
total_amount_spent    49950
Name: 0, dtype: object
